In [13]:
import pygad # https://pypi.org/project/pygad/
import sys
import math
import pandas as pd
import numpy as np


# 產生初始族群
def create_init_population():
    init_population = []
    for i in range(population_size):
        # 產生亂數
        random_num = list(np.random.permutation(num_gene))
        init_population.append(random_num)

        # 將亂數轉成 job number
        for j in range(num_gene):        
            init_population[i][j] = init_population[i][j] % num_job
    return init_population

# 紀錄修正前後的染色體
def set_solution_map(sol, repaired_sol):
    sol_str = "".join(str(x) for x in sol) # 串接陣列內各元素
    if sol_str in solution_map:
        pass
    else:
        solution_map[sol_str] = repaired_sol

# 校正染色體 (因交配後的染色體中，各 job 出現的次數可能不同)
def repair(solution):
    # print('before repairment')
    # print(solution)
    
    repaired_sol = solution.tolist()
    job_count = {}
    larger,less = [],[]

    for i in range(num_job):
        if i in repaired_sol:
            count = repaired_sol.count(i) # 染色體中，job i 出現的次數
            pos = repaired_sol.index(i) # 染色體中，job i 出現的位置
            job_count[i] = [count,pos]
        else:
            count = 0
            job_count[i] = [count, 0]
        
        if count > num_mc:
            larger.append(i) # 'larger' 紀錄染色體中，出現超過 num_mc 次的 job
        elif count < num_mc: # 'less' 紀錄染色體中，出現少於 num_mc 次的 job
            less.append(i)
            
    # 一一校正 lager 中各 job
    for k in range(len(larger)):
        chg_job = larger[k]
        while job_count[chg_job][0] > num_mc:
            for d in range(len(less)):
                if job_count[less[d]][0] < num_mc:
                    repaired_sol[job_count[chg_job][1]] = less[d]
                    # 更新 job_count
                    job_count[chg_job][1] = repaired_sol.index(chg_job)
                    job_count[chg_job][0] = job_count[chg_job][0] - 1
                    job_count[less[d]][0] = job_count[less[d]][0] + 1
                if job_count[chg_job][0] == num_mc:
                    break     
        
    # print('after repairment')
    # print(repaired_sol)
    set_solution_map(solution, repaired_sol)
    return repaired_sol

# 適合度函數
def fitness_func(solution, solution_idx):
    repaired_sol = repair(solution)
    
    # 計算適合度
    j_keys = [j for j in range(num_job)]
    key_count = { key: 0 for key in j_keys }
    j_count = { key: 0 for key in j_keys }
    m_keys = [j + 1 for j in range(num_mc)]
    m_count = { key: 0 for key in m_keys }
    '''
    j_keys
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    key_count
    {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
    j_count
    {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
    m_keys
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    m_count
    {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
    '''
    for i in repaired_sol:
        gen_t = int(pt[i][key_count[i]]) # job i 的第 key_count 次 processTime, pt[3][0] processTime=81. pt[4][0]=14==>job4 pt=81, job5 pt=14
        gen_m = int(ms[i][key_count[i]]) # job i 的第 key_count 次 機器 number
        j_count[i] = j_count[i] + gen_t # job i 的 累計 processTime
        m_count[gen_m] = m_count[gen_m] + gen_t # 機器 gen_m 的累計 processTime 
            
        # 把機台累計時間與job累計時間, 取最大時間, 更新機台/job最後時間
        if m_count[gen_m] < j_count[i]:
            m_count[gen_m] = j_count[i]
        elif m_count[gen_m] > j_count[i]:
            j_count[i] = m_count[gen_m]
            
        key_count[i] = key_count[i] + 1

    makespan = max(j_count.values()) # 完工時間 = 取最大的累計 processTime
    # print(makespan)
    return 1 / makespan # 目標: 完工時間最短





pt_tmp = pd.read_excel(r"d:\Projects\AI\POC\ClassMaterial\20210219_GA_生產排程案例\JSP_dataset.xlsx",sheet_name="Processing Time",index_col =[0])
ms_tmp = pd.read_excel(r"d:\Projects\AI\POC\ClassMaterial\20210219_GA_生產排程案例\JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col =[0])

num_mc = pt_tmp.shape[1] # machine 數
num_job = pt_tmp.shape[0] # job 數
num_gene = num_mc * num_job # 染色體內的基因數

pt = [list(map(int, pt_tmp.iloc[i])) for i in range(num_job)]
ms = [list(map(int,ms_tmp.iloc[i])) for i in range(num_job)]


population_size = 30
population_list = create_init_population()

solution_map = {} # key: source solution, value: repaired solution



# 建立 GA 實體
ga_instance = pygad.GA(
                       # 產生初始族群
                       initial_population = population_list,

                       # 選擇 (selection)
                       parent_selection_type = "rws", # 選擇方式
                       keep_parents = 2,

                       # 交配 (crossover)
                       num_parents_mating = population_size, # 取幾個個體進行交配                    
                       crossover_probability = 0.8, # 交配機率
                       crossover_type = "two_points", # 交配方式

                       # 突變 (mutation)
                       mutation_probability = 0.2, # 突變機率
                       mutation_type = "random", # 突變方式
                       mutation_by_replacement = True,
                       random_mutation_min_val = 0,
                       random_mutation_max_val = 9,

                       # 適應度函數
                       fitness_func = fitness_func,

                       num_generations = 100 # 跑幾個世代
                    )
ga_instance.run() # 執行 GA

# ga_instance.plot_result() # 繪製各世代的適應度趨勢





In [14]:
# 取得最佳解
solution, solution_fitness, solution_idx = ga_instance.best_solution()

# print("最佳解 : {solution}".format(solution = solution))
print("最佳解 : {solution}".format(solution = solution_map["".join(str(x) for x in solution)]))

print("最佳解的適應度 : {solution_fitness}".format(solution_fitness = solution_fitness))
print("最佳解的適應度倒數 : {solution_fitness}".format(solution_fitness = str(1/solution_fitness)))

if ga_instance.best_solution_generation != -1:
    print("最佳解落在第 {best_solution_generation} 個世代".format(best_solution_generation = ga_instance.best_solution_generation))

print("\n\n\n")

最佳解 : [3, 8, 5, 5, 1, 4, 9, 4, 9, 3, 5, 1, 0, 4, 3, 9, 8, 9, 3, 9, 5, 8, 6, 8, 6, 4, 1, 7, 3, 1, 7, 3, 4, 7, 8, 8, 9, 6, 1, 7, 5, 3, 2, 8, 5, 1, 1, 4, 0, 0, 4, 5, 2, 7, 7, 7, 3, 2, 5, 7, 0, 1, 6, 3, 8, 9, 0, 8, 1, 4, 9, 5, 6, 0, 7, 2, 2, 0, 6, 2, 9, 7, 6, 0, 2, 8, 6, 2, 4, 2, 0, 3, 6, 1, 6, 0, 2, 4, 5, 9]
最佳解的適應度 : 0.0008006405124099279
最佳解的適應度倒數 : 1249.0
最佳解落在第 17 個世代






In [15]:
 solution_map["".join(str(x) for x in solution)]

[3,
 8,
 5,
 5,
 1,
 4,
 9,
 4,
 9,
 3,
 5,
 1,
 0,
 4,
 3,
 9,
 8,
 9,
 3,
 9,
 5,
 8,
 6,
 8,
 6,
 4,
 1,
 7,
 3,
 1,
 7,
 3,
 4,
 7,
 8,
 8,
 9,
 6,
 1,
 7,
 5,
 3,
 2,
 8,
 5,
 1,
 1,
 4,
 0,
 0,
 4,
 5,
 2,
 7,
 7,
 7,
 3,
 2,
 5,
 7,
 0,
 1,
 6,
 3,
 8,
 9,
 0,
 8,
 1,
 4,
 9,
 5,
 6,
 0,
 7,
 2,
 2,
 0,
 6,
 2,
 9,
 7,
 6,
 0,
 2,
 8,
 6,
 2,
 4,
 2,
 0,
 3,
 6,
 1,
 6,
 0,
 2,
 4,
 5,
 9]

In [16]:
'''--------plot gantt chart-------'''
'''
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
'''
import pandas as pd
import chart_studio.plotly as py
import plotly.figure_factory as ff
import datetime

m_keys=[j+1 for j in range(num_mc)]  #[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
j_keys=[j for j in range(num_job)]   #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
key_count={key:0 for key in j_keys}  #{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
j_count={key:0 for key in j_keys}    #{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
m_count={key:0 for key in m_keys}    #{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
j_record={}

print(solution_map["".join(str(x) for x in solution)])
'''
job seq:sequence_best
[5, 0, 4, 7, 4, 6, 0, 0, 4, 5, 1, 5, 5, 5, 1, 6, 4, 3, 6, 6, 6, 4, 7, 3, 1, 1, 0, 5, 9, 6, 7, 9, 4, 0, 6, 8, 9, 4, 0, 3, 7, 8, 0, 8, 3, 5, 1, 9, 5, 2, 5, 3, 2, 1, 6, 1, 3, 8, 7, 2, 0, 9, 9, 7, 2, 6, 1, 9, 2, 3, 1, 2, 5, 2, 7, 8, 8, 6, 8, 3, 3, 3, 9, 2, 9, 7, 8, 4, 7, 7, 8, 9, 2, 0, 4, 4, 1, 2, 0, 8]
-->j4, j5, j10, j3, ...
-->m2, m3, m2, m2, ...
'''
solution_map_rp = solution_map["".join(str(x) for x in solution)]
for i in solution_map_rp:
    gen_t=int(pt[i][key_count[i]]) #pt[5][0] processTime=81. pt[0][0]=14==>job4 pt=81, job5 pt=14
    gen_m=int(ms[i][key_count[i]]) #ms[3][0] machine=2, ms[4][0]=3
    j_count[i]=j_count[i]+gen_t #job i的processTime
    m_count[gen_m]=m_count[gen_m]+gen_t #machine gen_m的processTime
    print(i,'processTime:',gen_t,'machine',gen_m) 
    #機台的時間與job的時間相比, 取最大 (因為可能須排隊)
    if m_count[gen_m]<j_count[i]:
        m_count[gen_m]=j_count[i]
    elif m_count[gen_m]>j_count[i]:
        j_count[i]=m_count[gen_m]
    
    start_time=str(datetime.timedelta(seconds=j_count[i]-pt[i][key_count[i]])) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=j_count[i]))
        
    j_record[(i,gen_m)]=[start_time,end_time]  #job i在機台m的時間區間
    
    key_count[i]=key_count[i]+1
    '''
    print(key_count)
    每個job算完processTime之後, 用key_count控制取下一次的job processTime
    key_count 
    {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0} --> 3: job4算完第1次pt, count+1
    {0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0} --> 4: job5算完第1次pt, count+1
    {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0} -->
    {0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1} -->...
    '''

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='Machine %s'%(m), Start='2018-07-14 %s'%(str(j_record[(j,m)][0])), Finish='2018-07-14 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j+1)))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Job shop Schedule')
#py.iplot(fig, filename='GA_job_shop_scheduling', world_readable=True) #-->SSL required
fig.show()

[3, 8, 5, 5, 1, 4, 9, 4, 9, 3, 5, 1, 0, 4, 3, 9, 8, 9, 3, 9, 5, 8, 6, 8, 6, 4, 1, 7, 3, 1, 7, 3, 4, 7, 8, 8, 9, 6, 1, 7, 5, 3, 2, 8, 5, 1, 1, 4, 0, 0, 4, 5, 2, 7, 7, 7, 3, 2, 5, 7, 0, 1, 6, 3, 8, 9, 0, 8, 1, 4, 9, 5, 6, 0, 7, 2, 2, 0, 6, 2, 9, 7, 6, 0, 2, 8, 6, 2, 4, 2, 0, 3, 6, 1, 6, 0, 2, 4, 5, 9]
3 processTime: 81 machine 2
8 processTime: 76 machine 1
5 processTime: 84 machine 3
5 processTime: 2 machine 2
1 processTime: 43 machine 1
4 processTime: 14 machine 3
9 processTime: 85 machine 2
4 processTime: 6 machine 1
9 processTime: 13 machine 1
3 processTime: 95 machine 3
5 processTime: 52 machine 6
1 processTime: 90 machine 3
0 processTime: 29 machine 1
4 processTime: 22 machine 2
3 processTime: 71 machine 1
9 processTime: 61 machine 3
8 processTime: 69 machine 2
9 processTime: 7 machine 7
3 processTime: 99 machine 5
9 processTime: 64 machine 9
5 processTime: 95 machine 4
8 processTime: 76 machine 4
6 processTime: 46 machine 2
8 processTime: 51 machine 6
6 processTime: 37 machine 1
4 